# Jupiter in Jupyter
### Smoothed Particle Hydrodynamics and Gas Giant Equilibrium 

Sai Chikine & Alex Rybchuk

Advisor: Professor Yecko

Ph 235

# Introduction - SPH


# Position Intialization

\begin{equation}

\end{equation}

# Density and Pressure Initializaion